In [1]:
#Basic Libraries
import numpy as np
import pandas as pd

#Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Text Handling Libraries
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [11]:
# recommendation
def get_tfid_recommendation(title, cosine_sim,df):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df['Description'].iloc[movie_indices]

In [46]:
df_clean = pd.read_csv('clean.csv')
df_clean['matnr'] = df_clean['matnr'].astype(str)
indices = pd.Series(df_clean.index, index=df_clean['Description']).drop_duplicates()
df_clean[['soup','Description']]

,soup,Description
0,dermaveen soap free all other non-therapeutic ...,DERMAVEEN SOAP FREEWASH500ML
1,kenhancer plaster topical antirheumaticssun ph...,KENHANCER PLASTER20'S
2,tecnis one zcb 06.5d cataract and anticataract...,TECNIS ONE ZCB 06.5D
3,redoxon kids da chew vitamin c combinationsbay...,REDOXON KIDS DA CHEWABLETAB60S
4,tecnis one zcb 21.0d cataract and anticataract...,TECNIS ONE ZCB 21.0D
...,...,...
4778,bepanthen first aid all other wound healing ag...,BEPANTHEN FIRST AIDCREAM30G FREE 3.5G
4779,(z) janumet xr 50/ dpp-iv inhibitor biguanide...,(Z) JANUMET XR 50/1000 TABS 56'S
4780,(z) mabthera vial other immunosuppressantsroch...,(Z) MABTHERA VIAL 500MG/50ML 1 MY
4781,(z) actemra vial other immunosuppressantsroche...,(Z) ACTEMRA VIAL 200MG/10ML 1 MY


In [47]:
df_clean.head()

,Unnamed: 0,matnr,zzprncode,zzibrnd,mtart,matkl,TherapeuticClass,PrincipalName,mstae,mstde,...,prat4,prat5,prat6,prat7,prat8,prat9,zzibrnd_new,TherapeuticClass_new,PrincipalName_new,soup
0,0,21047156,101276.0,DERMAVEEN SOAP FREE,ZF01,ZCS,All other non-therapeutic products,INOVA PHARMACEUTICALS (S) PTE,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['DERMAVEEN SOAP FREE'],['All other non-therapeutic products'],['INOVA PHARMACEUTICALS (S) PTE'],dermaveen soap free all other non-therapeutic ...
1,1,23046633,101234.0,KENHANCER PLASTER,ZF03,ZMC,Topical antirheumatics,SUN PHARMACEUTICAL SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['KENHANCER PLASTER'],['Topical antirheumatics'],['SUN PHARMACEUTICAL SDN BHD'],kenhancer plaster topical antirheumaticssun ph...
2,2,21210803,101223.0,TECNIS ONE ZCB 06.5D,ZF01,ZMD,Cataract and anticataractogenic preps.,ABBOTT MEDICAL OPTICS,LR,99910101,...,NaN,NaN,NaN,NaN,NaN,NaN,['TECNIS ONE ZCB 06.5D'],['Cataract and anticataractogenic preps.'],['ABBOTT MEDICAL OPTICS'],tecnis one zcb 06.5d cataract and anticataract...
3,3,23064902,101369.0,REDOXON KIDS DA CHEW,ZF03,ZCH,Vitamin C combinations,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['REDOXON KIDS DA CHEW'],['Vitamin C combinations'],['BAYER CO (MALAYSIA) SDN BHD'],redoxon kids da chew vitamin c combinationsbay...
4,4,21037570,101223.0,TECNIS ONE ZCB 21.0D,ZF01,ZMD,Cataract and anticataractogenic preps.,ABBOTT MEDICAL OPTICS,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['TECNIS ONE ZCB 21.0D'],['Cataract and anticataractogenic preps.'],['ABBOTT MEDICAL OPTICS'],tecnis one zcb 21.0d cataract and anticataract...


In [18]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_clean['soup'])
tfid_cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [59]:
searchstr = 'BEPANTHEN FIRST AIDCREAM30G FREE 3.5G'
idx = get_tfid_recommendation(searchstr,tfid_cosine_sim,df_clean).values
idx

array(['23064870', '23064880', '23064881', '23064920', '23064725',
       '23064727', '23064713', '23064705', '23064712', '23054980'],
      dtype=object)

In [41]:
df_clean.iloc[['4782']]

,Unnamed: 0,matnr,zzprncode,zzibrnd,mtart,matkl,TherapeuticClass,PrincipalName,mstae,mstde,...,prat4,prat5,prat6,prat7,prat8,prat9,zzibrnd_new,TherapeuticClass_new,PrincipalName_new,soup
4782,4782,23063718,101363.0,FOLTENE ANTI AGING,ZF03,ZCS,Other dermatological preparations,PARVUS SDN BHD,LR,20250120,...,NaN,NaN,NaN,NaN,NaN,NaN,['FOLTENE ANTI AGING'],['Other dermatological preparations'],['PARVUS SDN BHD'],foltene anti aging other dermatological prepar...


In [58]:
df_clean[df_clean['matnr'].isin(idx)]

,Unnamed: 0,matnr,zzprncode,zzibrnd,mtart,matkl,TherapeuticClass,PrincipalName,mstae,mstde,...,prat4,prat5,prat6,prat7,prat8,prat9,zzibrnd_new,TherapeuticClass_new,PrincipalName_new,soup
341,341,23064880,101369.0,BEPANTHEN,ZF03,ZPE,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['BEPANTHEN'],['All other wound healing agents'],['BAYER CO (MALAYSIA) SDN BHD'],bepanthen all other wound healing agentsbayer ...
633,633,23064705,101369.0,BEPANTHEN OINT 100G,ZF03,ZPE,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['BEPANTHEN OINT 100G'],['All other wound healing agents'],['BAYER CO (MALAYSIA) SDN BHD'],bepanthen oint 100g all other wound healing ag...
898,898,23064870,101369.0,BEPANTHEN FIRST AID,ZF03,ZPO,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250413,...,NaN,NaN,NaN,NaN,NaN,NaN,['BEPANTHEN FIRST AID'],['All other wound healing agents'],['BAYER CO (MALAYSIA) SDN BHD'],bepanthen first aid all other wound healing ag...
1407,1407,23064727,101369.0,BEPANTHEN OINTMENT 3,ZF03,ZPE,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['BEPANTHEN OINTMENT 3'],['All other wound healing agents'],['BAYER CO (MALAYSIA) SDN BHD'],bepanthen ointment 3 all other wound healing a...
1683,1683,23064713,101369.0,BEPANTHEN OINT 100G,ZF03,ZCH,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['BEPANTHEN OINT 100G'],['All other wound healing agents'],['BAYER CO (MALAYSIA) SDN BHD'],bepanthen oint 100g all other wound healing ag...
1752,1752,23064920,101369.0,BEPANTHEN BABY BATH,ZF03,ZCS,Wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['BEPANTHEN BABY BATH'],['Wound healing agents'],['BAYER CO (MALAYSIA) SDN BHD'],bepanthen baby bath wound healing agentsbayer ...
1931,1931,23054980,101271.0,COMFORT AID GEL BREA,ZF03,ZMD,All other Therapeutic Products,SOZA HEALTHCARE SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['COMFORT AID GEL BREA'],['All other Therapeutic Products'],['SOZA HEALTHCARE SDN BHD'],comfort aid gel brea all other therapeutic pro...
2406,2406,23064725,101369.0,BEPANTHEN OINTMENT 3,ZF03,ZPE,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['BEPANTHEN OINTMENT 3'],['All other wound healing agents'],['BAYER CO (MALAYSIA) SDN BHD'],bepanthen ointment 3 all other wound healing a...
3096,3096,23064881,101369.0,BEPANTHEN,ZF03,ZPE,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['BEPANTHEN'],['All other wound healing agents'],['BAYER CO (MALAYSIA) SDN BHD'],bepanthen all other wound healing agentsbayer ...
3158,3158,23064712,101369.0,BEPANTHEN OINT 30G V,ZF03,ZCH,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,['BEPANTHEN OINT 30G V'],['All other wound healing agents'],['BAYER CO (MALAYSIA) SDN BHD'],bepanthen oint 30g v all other wound healing a...
